## UFEA 7주차 과제(ELS Pricing by using MC)

### ELS

ELS: Equity Linked Securities

### Monte Calro Simulation

①   주가 패스를 생성한다.

②   생성된 주가 패스에 대해서 상품의 페이오프를 계산한다. 이 페이오프를 현재 계산시점으로 할인한다.

③   ①~②번을 여러번 반복하여 여러 개의 할인 페이오프 샘플을 얻는다.

④   ③에서 얻은 샘플들의 기댓값을 구한다.

스텝다운 ELS은 낙인 여부를 관찰해야 하니 데일리로 주가 패스를 만들어야겠다!
 

라는 생각이 듭니다. 낙인을 치고 만기까지 질질 끌다가 마지막 배리어를 넘지 못하여 손실 상환이 되는 경우가 있고, 이 부분을 처리하기 위해 낙인 여부를 판단해야 하므로 데일리 주가를 생성해내는 게 맞아 보입니다.

### Code

In [1]:
import numpy as np

import time

### 1-star ELS(Monte Carlo)

In [17]:
class Market:	#market parameter를 관리하기 위한 class 도입
    def __init__(self, rfr):
        self._rfr = rfr	#class 멤버 변수 rfr : 무위험 이자율을 뜻함

In [18]:
class Underlying:	#Underlying class 설정
    def __init__(self, refprice, spotvalue, volatility, dividend):
        self._volatility = volatility	#멤버변수 _volatility : 변동성
        self._dividend = dividend		#멤버변수 _dividend : 연속 배당률
        self._spot = spotvalue			#멤버변수 _spot : 기초자산 현재가
        self._refprice = refprice		#멤버변수 _refprice : 기초자산 기준가

In [21]:

def one_dimension_mc(objUnderlying: Underlying, objMarket: Market, redemption_schedule, coupon,
                 full_dummy, barrier, ki_barrier, n_iteration):
# 입력파라미터
# objUnderlying : 기초자산 class
# objMarket : 시장 파라미터 class
# redemption_schedule : 상환스케줄 array (발행시점부터 경과된 일자, 예를들어 
# 6m = 125, 1y=250,..3y=750  -> (125, 250, 375, 500, 625, 750))
# coupon : 상환시점에 수익상환시 지급되는 쿠폰 array (예컨대 {5%, 10%, 15%, 20%, 25%, 30%)
# full_dummy : 만기시 full dummy 쿠폰 값
# barrier : 상환시점의 배리어 array ( 예컨대, (0.9, 0.9, 0.85, 0.85, 0.8, 0.8))
# ki_barrier : 낙인 배리어
# n_iteration : MonteCarlo Simulation 횟수

    #class 멤버변수에서 얻어온 변동성
    vol = objUnderlying._volatility	
    div = objUnderlying._dividend
    rfr = objMarket._rfr
    current_spot = objUnderlying._spot
    refprice = objUnderlying._refprice
    
    #하루 단위를 연단위로 표시
    dt = 1 / 250

    drift = (rfr - div - 0.5 * vol ** 2) * dt	#일간 drift term
    diffusion = vol * np.sqrt(dt)				#일간 diffusion term
    maturity = redemption_schedule[-1]			#상환스케쥴의 마지막 원소는 만기일자

    sum_price = 0	# MC시뮬레이션 하나당 발생하는 결과를 누적할 변수

    redemp_prob = np.zeros(len(redemption_schedule) + 2)	
    # 각 상환시점 쿠폰 상환 확률, full_dummy 지급확률, 손실상환 확률을 구할 변수
    np.random.seed(0)	# random number 시드고정
    start_time = time.time()	#계산소요시간을 산출하기 위해 start time 기록
    for i in range(n_iteration):    # simulation을 돌리기 시작
        temp_price = 0              # 각 simulation 당 할인된 페이오프 저장할 변수
        sSeries = [current_spot]    # sSeries라는 list에 우선 현재가 집어넣음
        payoff = 0                  # payoff 변수
        discount_factor = 0         # 할인팩터 변수
        old_spot = current_spot     # 재귀적으로 일일주가패스 생성을 위한 변수 old_spot도입
        for j in range(maturity):
            new_spot = old_spot * np.exp(drift + diffusion * np.random.normal())
            # GBM model로 내일의 주가 생성
            sSeries.append(new_spot)    # 새로이 생성된 new_spot 을 sSeries list에 삽입
            old_spot = new_spot         # 재귀적 생성을 위해 다시 old_spot에 대입
        sSeries = np.array(sSeries) / refprice	# 퍼포먼스 계산을 위해 기준가 refprice로 나눔

        for k in range(len(redemption_schedule)):  # 상환스케쥴 배열 for문을 돌리며
            ind = redemption_schedule[k]           # k번째 상환스케쥴 일자를 ind에 저장
            redemption_mat = ind                   # 저장된 값이 k번째 상환만기일자가 됨
            if sSeries[ind] >= barrier[k]:         # 딱 그날 퍼포먼스가 배리어 상회하면
                payoff = 1 + coupon[k]             # 수익쿠폰 및 원금 상환
                discount_factor = np.exp(-rfr * redemption_mat / 250)	# 할인펙터 계산
                redemp_prob[k] += 1                # k번째 상환시점에 상환됐음을 count함
                break                              # 상품이 상환되어 끝났으므로 for문을 끝냄
            else:	# 쿠폰 수익 상환이 안되었다면,
                discount_factor = np.exp(-rfr * redemption_mat / 250)	#만기까지 간 상황, 할인팩터 구함
                if (k == len(redemption_schedule) - 1) & (min(sSeries) < ki_barrier):
                # 만기상환시점이고, 주가패스 중 최저점이 낙인배리어보다 아래면(즉, 낙인 발생)
                    payoff = sSeries[redemption_mat]	# 만기퍼포먼스가 payoff가 됨
                    redemp_prob[-1] += 1				# redemp_prob의 마지막 원소에 counting함
                elif (k == len(redemption_schedule) - 1) & (min(sSeries) >= ki_barrier):
                #만기상한시점이고 낙인을 안쳤으면
                    payoff = 1 + full_dummy	#full dummy지급구간임
                    redemp_prob[-2] += 1        #마지막에서 두번째 원소가 full_dummy 지급 counting

        temp_price = payoff * discount_factor   #할인된 페이오프
        sum_price += temp_price                 #할인된 페이오프를 누적함

    els_value = sum_price / n_iteration         #누적한 결과를 시뮬레이션 횟수로 나눔, 즉 기댓값
    cal_time = round(time.time() - start_time, 3)	#계산 종료, 계산경과시간 계산
    redemp_prob /= n_iteration	#redemp_prob 는counting만 한 것이므로 simul횟수로 나눠 확률을 구해줌
    return els_value, cal_time, redemp_prob	# elsvalue와 계산시간, 상환확률 return

In [22]:
underlying_spot = Underlying(refprice=100, spotvalue=100, volatility=0.3, dividend=0)
# 기준가 100, 현재가격 100, 변동성 30%, 배당 0인 기초자산
interest_rate = Market(0.03)
# 이자율이 3%인 시장 파라미터 클래스
redemption_schedule = np.array([1, 2, 3, 4, 5, 6]) * 125    #3y 6m, 6 Chance Stepdown ELS
coupon = np.array([1, 2, 3, 4, 5, 6]) * 0.05                # coupon : 10% p.a.
full_dummy = coupon[-1]	                                    # full dummy cpn = 30%
barrier = np.array([0.9, 0.9, 0.85, 0.85, 0.8, 0.8])        # 조기/만기상환 배리어
ki_barrier = 0.6                                            # knock in barrier
n_iteration = 10000                                         # simulation 횟수

els_price, cal_time, prob = one_dimension_mc(underlying_spot, interest_rate, redemption_schedule, coupon,
                                         full_dummy, barrier, ki_barrier, n_iteration)
np.set_printoptions(suppress=True)   #scientific notation 을 피하기 위한 print 옵션
print(els_price)
print(cal_time)
print(prob)

0.9833693017422173
8.858
[0.6798 0.0886 0.0557 0.0237 0.0231 0.0131 0.0016 0.1144]


#### Brown Bridge(Monte Carlo)

In [30]:
def one_dimension_mc_bb(objUnderlying: Underlying, objMarket: Market, redemption_schedule, coupon,
                    full_dummy, barrier, ki_barrier, n_iteration):
    vol = objUnderlying._volatility
    div = objUnderlying._dividend
    rfr = objMarket._rfr
    current_spot = objUnderlying._spot
    refprice = objUnderlying._refprice
    dt = 1 / 250
    drift = (rfr - div - 0.5 * vol ** 2)
    maturity = redemption_schedule[-1]

    sum_price = 0
    temp_price = 0

    nSchedule = len(redemption_schedule)      # nSchedule : 상환스케쥴 개수 (6개) 

    tenor = np.zeros(nSchedule)               # tenor는 다음상환시점까지의 잔존시간 
    tenor[0] = redemption_schedule[0]         # 현재시점은 0이므로 처음 조기상환시점 T1까지 잔존시간 T1
    tenor[1:] = np.diff(redemption_schedule)  # T_{i-1}과 T_i 사이의 잔존시간은 T_i - T_{i-1}
    tenor /= 250                              # redemption_schedule의 day개념이므로 연으로 환산
    redemp_prob = np.zeros(nSchedule + 2)     # 각 쿠폰상환 확률(1~6) 및 fulldummy, 손실상환 확률 arr
    np.random.seed(0)
    sum_payoff = 0

    start_time = time.time()
    for i in range(n_iteration):    # 시뮬레이션을 n_iteration회 돌린다.
        rn = []                     # 각 조기/만기상한 시점의 주가를 생성해 낸 random 변수를 저장할 list
        redem_spot = []             # 각 조기/만기상한 시점의 주가를 저장할 list
        old_spot = current_spot     # 현재가를 old_spot 변수에 저장
        ki_occurs = False           # ki_occurs는 boolean type으로 낙인이 일어났는지 여부
        redem_spot.append(old_spot) # redem_spot에 현재가를 우선 저장

        for j in range(nSchedule):                # Schedule 개수만큼 for문을 돌리며
            rn_normal = np.random.normal()        # 정규분포 난수 한개 발생시킴
            new_spot = old_spot * np.exp(drift * tenor[j] + vol * np.sqrt(tenor[j]) * rn_normal)
                                                  # 재귀적으로 새로운 주가 생성
                                                  # 재귀적 산출을 위해 {T_i -T_{i-1}} 를 구한 것
            if new_spot / refprice >= barrier[j]: # next 상환시점 주가 퍼포먼스가 배리어를 상회하면
                payoff = 1 + coupon[j]            # 원금 및 쿠폰지급
                df = np.exp(-rfr * redemption_schedule[j] / 250)
                                                  # 할인팩터
                redemp_prob[j] += 1               # 그 상환시점을 counting하여 확률구할때 씀
                break       
            else:
                old_spot = new_spot          # 쿠폰 상환이 안된 상태(break 가 안먹음)
                redem_spot.append(new_spot)  # 상환시점의 주가들을 모아놓은 list
                if new_spot / refprice < ki_barrier:  
                                             # 만일 상환시점 주가가 ki barrier 밑이면
                    ki_occurs = True         # ki flag를 true로 설정
                rn.append(rn_normal)
                if j == nSchedule - 1:       # 만기 시점일 때,
                    df = np.exp(-rfr * redemption_schedule[j] / 250)
                                             # 할인팩터 구함
                    if ki_occurs:            # ki flag 가 true이면,
                        payoff = old_spot / refprice 
                                             # 주가 퍼포먼스에 연동된 payoff 지급
                        redemp_prob[-1] += 1 # 손실상환 counting은 redemp_prob 제일 마지막 원소에
                    else:                    # ki flag가 false이면,
                                             # 즉 상환시점의 주가상태는 ki 친 상황이 아니면
                                             # 상환시점 사이에서 knock in을 쳤는지 알아봐야 한다!
                        for k in range(nSchedule):
                            # Brownian Bridge를 만들기 위해, 두 시점과 두 시점의 normal random
                            # 변수를 구함
                            if k == 0:
                                t1, t2 = 0, redemption_schedule[k]
                                s1, s2 = redem_spot[k], redem_spot[k + 1]
                                zt1, zt2 = 0, (np.log(s2 / refprice) - drift * np.sqrt(t2)) / (vol * np.sqrt(t2))
                            else:
                                t1, t2 = redemption_schedule[k - 1], redemption_schedule[k]
                                zt1 = (np.log(s1 / refprice) - drift * np.sqrt(t1)) / (vol * np.sqrt(t1))
                                zt2 = (np.log(s2 / refprice) - drift * np.sqrt(t2)) / (vol * np.sqrt(t2))

                            wiener = np.zeros(t2 - t1 + 1)  
                            timegrid = np.arange(t2 - t1 + 1)
                            for c in range(t2 - t1):   # 두 점을 연결하기 위한 wiener process설계
                                wiener[c + 1] = wiener[c] + np.random.normal() * np.sqrt(1 / 250)
                            bbridge = zt1 + timegrid / (t2 - t1) * (zt2 - zt1) + wiener - timegrid / (t2 - t1) * wiener[
                                -1]                    # Brownian bridge 건설
                            svec = s1 * np.exp(drift * dt + vol * np.sqrt(dt) * bbridge)
                                                       # Brownian bridge로 일일주가 생성
                            if min(svec) / refprice < ki_barrier: # 생성된 주가패스의 최저값이 ki barrier 아래면
                                ki_occurs = True                  # 낙인 flag = True
                                break                             # 이미 낙인 상황이므로 더 볼 필요도 없음
                        if ki_occurs:
                            payoff = old_spot / refprice          # 낙인쳤으면 손실상환
                            redemp_prob[-1] += 1                  # 낙인상황 counting
                        else:
                            payoff = 1 + full_dummy               # 낙인안쳤으면 full dummy 상환
                            redemp_prob[-2] += 1                  # full dummy 지급상황 counting
        sum_payoff += df * payoff
    
    cal_time = round(time.time() - start_time, 3)
    els_value = sum_payoff / n_iteration
    redemp_prob /= n_iteration
    return els_value, cal_time, redemp_prob                                 # 할인 페이오프 누적

In [32]:
underlying_spot = Underlying(refprice=100, spotvalue=100, volatility=0.3, dividend=0)
# 기준가 100, 현재가격 100, 변동성 30%, 배당 0인 기초자산 객체 설계
interest_rate = Market(0.03)
# 이자율이 3%인 시장 파라미터 클래스 설계
redemption_schedule = np.array([1, 2, 3, 4, 5, 6]) * 125    #3y 6m, 6 Chance Stepdown ELS
coupon = np.array([1, 2, 3, 4, 5, 6]) * 0.05                # coupon : 10% p.a.
full_dummy = coupon[-1]	                                    # full dummy cpn = 30%
barrier = np.array([0.9, 0.9, 0.85, 0.85, 0.8, 0.8])        # 조기/만기상환 배리어
ki_barrier = 0.6                                            # knock in barrier
n_iteration = 10000                                         # simulation 횟수

els_price, cal_time, prob = one_dimension_mc(underlying_spot, interest_rate, redemption_schedule, coupon,
                                         full_dummy, barrier, ki_barrier, n_iteration)
els_price_bb, cal_time_bb, prob_bb = one_dimension_mc_bb(underlying_spot, interest_rate, redemption_schedule, coupon,
                                                     full_dummy, barrier, ki_barrier, n_iteration)
np.set_printoptions(suppress=True)

print('----------')
print(els_price)
print(cal_time)
print(prob)
print('----------')
print(els_price_bb)
print(cal_time_bb)
print(prob_bb)
print('----------')

----------
0.9833693017422173
8.828
[0.6798 0.0886 0.0557 0.0237 0.0231 0.0131 0.0016 0.1144]
----------
0.9882142191996907
0.17
[0.6796 0.0905 0.0514 0.0225 0.0242 0.0137 0.0118 0.1063]
----------


In [44]:
underlying_spot = Underlying(refprice=100, spotvalue=100, volatility=0.3, dividend=0)
# 기준가 100, 현재가격 100, 변동성 30%, 배당 0인 기초자산 객체 설계
interest_rate = Market(0.03)
# 이자율이 3%인 시장 파라미터 클래스 설계
redemption_schedule = np.array([1, 2, 3, 4, 5, 6]) * 125    #3y 6m, 6 Chance Stepdown ELS
coupon = np.array([1, 2, 3, 4, 5, 6]) * 0.05                # coupon : 10% p.a.
full_dummy = coupon[-1]	                                    # full dummy cpn = 30%
barrier = np.array([0.9, 0.9, 0.85, 0.85, 0.8, 0.8])        # 조기/만기상환 배리어
ki_barrier = 0.6                                            # knock in barrier
n_iteration = 100000

els_price_bb, cal_time_bb, prob_bb = one_dimension_mc_bb(underlying_spot, interest_rate, redemption_schedule, coupon,
                                                     full_dummy, barrier, ki_barrier, n_iteration)
np.set_printoptions(suppress=True)                                         # simulation 횟수

print('----------')
print(els_price_bb)
print(cal_time_bb)
print(prob_bb)
print('----------')

----------
0.9878295701967492
1.607
[0.67946 0.08792 0.05345 0.02498 0.02344 0.01316 0.01153 0.10606]
----------


### Two-star ELS

In [14]:
class Market_2: #market prarmeter를 관리하기 위한 class
    def __init__(self, rfr, correlation):
        self._rfr = rfr
        self._correlation = correlation

In [15]:
class Underlying_2:
    def __init__(self,refprice, spotvalue, volatility, dividend):
        self._refprice = refprice #기초자산 기준가
        self._spotvalue = spotvalue #기초자산 현재가
        self._volatility = volatility #변동성
        self._dividend = dividend #연속 배당률

In [17]:
def two_dimension_mc(obj_underlying_1: Underlying_2, obj_underlying_2: Underlying_2, obj_market: Market_2, 
                     redemption_schedule, coupon, full_dummy, barrier, ki_barrier, n_iteration):
    
    # 입력파라미터
    # obj_underlying_1 : 기초자산_1 class
    # obj_underlying_2 : 기초자산_2 class
    # objMarket : 시장 파라미터 class
    # redemption_schedule : 상환스케줄 array (발행시점부터 경과된 일자, 예를들어 
    # 6m = 125, 1y=250,..3y=750  -> (125, 250, 375, 500, 625, 750))
    # coupon : 상환시점에 수익상환시 지급되는 쿠폰 array (예컨대 {5%, 10%, 15%, 20%, 25%, 30%)
    # full_dummy : 만기시 full dummy 쿠폰 값
    # barrier : 상환시점의 배리어 array ( 예컨대, (0.9, 0.9, 0.85, 0.85, 0.8, 0.8))
    # ki_barrier : 낙인 배리어
    # n_iteration : MonteCarlo Simulation 횟수
    
    vol_1 = obj_underlying_1._volatility
    div_1 = obj_underlying_1._dividend
    current_spot_1 = obj_underlying_1._spotvalue
    refprice_1 = obj_underlying_1._refprice
    
    vol_2 = obj_underlying_2._volatility 
    div_2 = obj_underlying_2._dividend
    current_spot_2 = obj_underlying_2._spotvalue
    refprice_2 = obj_underlying_2._refprice

    rfr = obj_market._rfr
    corr = obj_market._correlation

    dt = 1/250
    maturity = redemption_schedule[-1]

    drift_1 = (rfr - div_1 -.5*vol_1**2)
    diffusion_1 = vol_1 * np.sqrt(dt)
    drift_2 = (rfr - div_2 -.5*vol_2**2)
    diffusion_2 = vol_2 * np.sqrt(dt)

    np.random.seed(0) # random number 시드고정
    sum_price = 0 # MC시뮬레이션 하나당 발생하는 결과를 누적할 변수
    redemp_prob = np.zeros(len(redemption_schedule) + 2) # 각 상환시점 쿠폰 상환 확률, full_dummy 지급확률, 손실상환 확률을 구할 변수
    start_time = time.time() #계산소요시간을 산출하기 위해 start time 기록

    for i in range(n_iteration):
        temp_price = 0
        payoff = 0
        discount_factor = 0
        s_series_1 = [current_spot_1]
        s_series_2 = [current_spot_2]
        old_spot_1 = current_spot_1
        old_spot_2 = current_spot_2

        for j in range(maturity):
            Z1 = np.random.standard_normal()
            Z2 = corr * Z1 + (1-corr**2) * np.random.standard_normal()

            new_spot_1 = old_spot_1 * np.exp(drift_1 + diffusion_1 * Z1)
            s_series_1.append(new_spot_1)
            old_spot_1 = new_spot_1

            new_spot_2 = old_spot_2 * np.exp(drift_2 + diffusion_2 * Z2)
            s_series_2.append(new_spot_2)
            old_spot_2 = new_spot_2

        s_series_1 = np.array(s_series_1)/refprice_1
        s_series_2 = np.array(s_series_2)/refprice_2

        for k in range(len(redemption_schedule)):
            idx = redemption_schedule[k]
            redemption_mat = idx
            if (s_series_1[idx] >= barrier[k]) and (s_series_2[idx] >= barrier[k]):
                payoff = 1 + coupon[k]
                discount_factor = np.exp(-rfr * redemption_mat/250)
                redemp_prob[k] += 1
                break
            else:
                discount_factor = np.exp(-rfr * redemption_mat/250)
                if (k==len(redemption_schedule)-1) & ((min(s_series_1)<ki_barrier) or (min(s_series_2)<ki_barrier)):
                    payoff = min(s_series_1[redemption_mat], s_series_2[redemption_mat])
                    redemp_prob[-1] += 1
                elif (k==len(redemption_schedule)-1) & ((min(s_series_1)>=ki_barrier) or (min(s_series_2)>=ki_barrier)):
                    payoff = 1 + full_dummy
                    redemp_prob[-2] += 1

        temp_price = payoff * discount_factor
        sum_price += temp_price

    els_value = sum_price/n_iteration

    cal_time = round(time.time() - start_time, 3)
    redemp_prob /= n_iteration
    return els_value, cal_time, redemp_prob


In [18]:
# 기초자산 1
refprice_1 = 100
spotvalue_1 = 100
volatility_1 = 0.2
dividend_1 = 0.01

#기초자산 2
refprice_2 = 100
spotvalue_2 = 100
volatility_2 = 0.4
dividend_2 = 0.05

underlying_1 = Underlying_2(refprice=refprice_1, spotvalue=spotvalue_1, volatility=volatility_1, dividend=dividend_1)
underlying_2 = Underlying_2(refprice=refprice_2, spotvalue=spotvalue_2, volatility=volatility_2, dividend=dividend_2)

correlation = 0.5
market = Market_2(rfr = 0.03, correlation=0.5)

redemption_schedule = np.array([1,2,3,4,5,6]) * 125 # 3y, 6m, 6 Chances stepdown ELS
coupon = np.array([1,2,3,4,5,6]) * 0.05
full_dummy = coupon[-1]
barrier = np.array([0.9, 0.9, 0.85, 0.8, 0.75, 0.70])
ki_barrier = 0.5
n_iteration = 10000

In [19]:
els_price, cal_time, prob = two_dimension_mc(underlying_1, underlying_2, market, redemption_schedule, coupon, 
                                             full_dummy, barrier, ki_barrier, n_iteration)

np.set_printoptions(suppress=True)
print(els_price)
print(cal_time)
print(prob)


2.9907876584476392e-33
13.482
[0. 0. 0. 0. 0. 0. 0. 1.]
